In [21]:
import torch

print(f"Is CUDA supported by this system? {torch.cuda.is_available()}")
print(f"CUDA version: {torch.version.cuda}")

DEVICE = torch.device("cuda:1")
torch.cuda.set_device(DEVICE)
print(DEVICE)

Is CUDA supported by this system? True
CUDA version: 12.1
cuda:1


In [22]:
!pip install transformers
!pip install evaluate
!pip install rouge
!pip install spacy


import torch
import json
from tqdm import tqdm
import torch.nn as nn
from torch.optim import Adam
import nltk
import spacy
import string
import evaluate  # Bleu
from torch.utils.data import Dataset, DataLoader, RandomSampler
import pandas as pd
import numpy as np
import transformers
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from transformers import T5Tokenizer, T5Model, T5ForConditionalGeneration, T5TokenizerFast

import warnings
warnings.filterwarnings("ignore")

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [23]:
def load_dataset(file_name):
    import pandas as pd
    import json
    
    df = []
    df_multi = []
    df_phrase = []
    df_passage = []
    with open('/DATA1/ai20btech11028/vojes_nlp/Data/' + file_name, encoding='utf-8') as f:
        for i in f:
            i = json.loads(i)
                
            tweet = i['postText']
            article_title = i['targetTitle']
            article = ' '.join(i['targetParagraphs'])
            spoilers = i['spoiler']
            label = i['tags']
            keywords = i['targetKeywords']
            positions = i['spoilerPositions']
            
            assert len(tweet) == 1
            tweet = tweet[0]
            
            assert len(label) == 1
            label = label[0]
            
            if label not in ['phrase', 'phrases', 'passage', 'multi']:
                print(label)
                
            assert label in ['phrase', 'phrases', 'passage', 'multi']
            
            if label == 'phrase' or label == 'phrases':
                label = 0
            elif label == 'multi':
                label = 2
            else:
                label = 1
                
            
            df += [{'question': tweet , 'context': i['targetParagraphs'], 'article': article_title + article, 'spoiler':spoilers,
                    'labels': label, 'keywords':keywords, 'spoilerPositions':positions}]

    return pd.DataFrame(df)
            
    
# # test_dataset = load_dataset('test.jsonl')
train_dataset = load_dataset('train.jsonl')
validation_dataset = load_dataset('validation.jsonl')
# print(train_dataset)

In [24]:
from datasets import load_dataset

dataset = load_dataset("MateuszW/generated_questions")

genQue_train = dataset['train']['generated_questions']
genQue_test = dataset['validation']['generated_questions']

In [25]:
train_dataset["generated_que"] = list(genQue_train)
validation_dataset["generated_que"] = list(genQue_test)
train_dataset

,question,context,article,spoiler,labels,keywords,spoilerPositions,generated_que
0,"Wes Welker Wanted Dinner With Tom Brady, But P...",[It’ll be just like old times this weekend for...,"Wes Welker Wanted Dinner With Tom Brady, But P...",[how about that morning we go throw?],1,"new england patriots, ricky doyle, top stories,","[[[3, 151], [3, 186]]]",What did Tom Brady do instead of having dinner...
1,NASA sets date for full recovery of ozone hole,[2070 is shaping up to be a great year for Mot...,Hole In Ozone Layer Expected To Make Full Reco...,[2070],0,"ozone layer,ozone hole determined by weather,M...","[[[0, 0], [0, 4]]]",What is the date that NASA has set for the ful...
2,This is what makes employees happy -- and it's...,"[Despite common belief, money isn't the key to...",Intellectual Stimulation Trumps Money For Empl...,[intellectual stimulation],0,"employee happiness money,employee happiness in...","[[[1, 186], [1, 210]]]",What makes employees happy?
3,Passion is overrated — 7 work habits you need ...,"[It’s common wisdom. Near gospel really, and n...","‘Follow your passion’ is wrong, here are 7 hab...",[Purpose connects us to something bigger and i...,2,"business, work-life, careers","[[[11, 25], [11, 101]], [[17, 56], [17, 85]], ...",What are the 7 work habits that are considered...
4,The perfect way to cook rice so that it's perf...,"[Boiling rice may seem simple, but there is a ...",Revealed: The perfect way to cook rice so that...,[in a rice cooker],0,"Quora,users,share,perfect,way,cook,rice","[[[5, 60], [5, 76]]]",What is the best way to cook rice so that it's...
...,...,...,...,...,...,...,...,...
3195,Has Facebook's video explosion completely shak...,"[A long time ago in a galaxy far, far away...W...","Facebook Video Surging, But YouTube Still Offe...",[it hasn’t necessarily taken the wind out of Y...,1,"Facebook,web video,web video ads,YouTube","[[[7, 50], [7, 118]]]",Has Facebook's video explosion completely shak...
3196,Cop Is Eating At A Chili's When Teen Hands Him...,"[The Kansas City, Kansas Police Department are...",Cop is eating at Chili's when teen hands him f...,"[It read, ""Thanks for keeping us safe.""]",1,None,"[[[0, 317], [0, 355]]]",What did the teen give the cop?
3197,5 popular myths about visible signs of aging t...,"[Obama looks decades younger already, but what...",5 popular myths about visible signs of aging t...,[1. Anti-wrinkle creams will erase the fine li...,2,,"[[[6, 0], [6, 73]], [[10, 0], [10, 109]], [[14...",What are the 5 popular myths about visible sig...
3198,You need to see this Twitter account that pred...,"[What the HELL?!, 1. Unless you’re somewhere w...","WTF, It Looks Like This Twitter Account ""Predi...",[@beyoncefan666],0,,"[[[3, 55], [3, 69]]]",What is the purpose of the Twitter account men...


In [26]:
TOKENIZER = T5TokenizerFast.from_pretrained("t5-base")
MODEL = T5ForConditionalGeneration.from_pretrained("t5-base", return_dict=True).to(DEVICE)
OPTIMIZER = Adam(MODEL.parameters(), lr=0.0001)
Q_LEN = 256   # Question Length
T_LEN = 128    # Target Length
BATCH_SIZE = 16
DEVICE = "cuda:1"

In [27]:
# Extracting context, question, and answers from the dataset

def prepare_data(train_dataset):
    articles = []
    
    for i in range(len(train_dataset)):
        
        clickbait = train_dataset.iloc[i]
        context = clickbait['article']
        question = clickbait['generated_que']
        answer = ' '.join(clickbait['spoiler'])

        inputs = {"context": context, "question": question, "answer": answer}

        articles.append(inputs)

    return articles

In [28]:
data = prepare_data(train_dataset)

# Create a Dataframe
data = pd.DataFrame(data)

In [29]:
class QA_Dataset(Dataset):
    def __init__(self, tokenizer, dataframe, q_len, t_len):
        self.tokenizer = tokenizer
        self.q_len = q_len
        self.t_len = t_len
        self.data = dataframe
        self.questions = self.data["question"]
        self.context = self.data["context"]
        self.answer = self.data['answer']
        
    def __len__(self):
        return len(self.questions)
    
    def __getitem__(self, idx):
        question = self.questions[idx]
        context = self.context[idx]
        answer = self.answer[idx]
        
        question_tokenized = self.tokenizer(question, context, max_length=self.q_len, padding="max_length",
                                                    truncation=True, pad_to_max_length=True, add_special_tokens=True)
        answer_tokenized = self.tokenizer(answer, max_length=self.t_len, padding="max_length", 
                                          truncation=True, pad_to_max_length=True, add_special_tokens=True)
        
        labels = torch.tensor(answer_tokenized["input_ids"], dtype=torch.long)
        labels[labels == 0] = -100
        
        return {
            "input_ids": torch.tensor(question_tokenized["input_ids"], dtype=torch.long),
            "attention_mask": torch.tensor(question_tokenized["attention_mask"], dtype=torch.long),
            "labels": labels,
            "decoder_attention_mask": torch.tensor(answer_tokenized["attention_mask"], dtype=torch.long)
        }

In [30]:
# Dataloader

train_data, val_data = train_test_split(data, test_size=0.2, random_state=42)

train_sampler = RandomSampler(train_data.index)
val_sampler = RandomSampler(val_data.index)

qa_dataset = QA_Dataset(TOKENIZER, data, Q_LEN, T_LEN)

train_loader = DataLoader(qa_dataset, batch_size=BATCH_SIZE, sampler=train_sampler)
val_loader = DataLoader(qa_dataset, batch_size=BATCH_SIZE, sampler=val_sampler)

In [31]:
train_loss = 0
val_loss = 0
train_batch_count = 0
val_batch_count = 0

for epoch in range(5):
    MODEL.train()
    for batch in tqdm(train_loader, desc="Training batches"):
        input_ids = batch["input_ids"].to(DEVICE)
        attention_mask = batch["attention_mask"].to(DEVICE)
        labels = batch["labels"].to(DEVICE)
        decoder_attention_mask = batch["decoder_attention_mask"].to(DEVICE)

        outputs = MODEL(
                          input_ids=input_ids,
                          attention_mask=attention_mask,
                          labels=labels,
                          decoder_attention_mask=decoder_attention_mask
                        )

        OPTIMIZER.zero_grad()
        outputs.loss.backward()
        OPTIMIZER.step()
        train_loss += outputs.loss.item()
        train_batch_count += 1
    
    #Evaluation
    MODEL.eval()
    for batch in tqdm(val_loader, desc="Validation batches"):
        input_ids = batch["input_ids"].to(DEVICE)
        attention_mask = batch["attention_mask"].to(DEVICE)
        labels = batch["labels"].to(DEVICE)
        decoder_attention_mask = batch["decoder_attention_mask"].to(DEVICE)

        outputs = MODEL(
                          input_ids=input_ids,
                          attention_mask=attention_mask,
                          labels=labels,
                          decoder_attention_mask=decoder_attention_mask
                        )

        OPTIMIZER.zero_grad()
        outputs.loss.backward()
        OPTIMIZER.step()
        val_loss += outputs.loss.item()
        val_batch_count += 1
        
    print(f"{epoch+1}/{10} -> Train loss: {train_loss / train_batch_count}\tValidation loss: {val_loss/val_batch_count}")

Validation batches: 100%|███████████████████████████████████████████████████████████████| 40/40 [00:14<00:00,  2.74it/s]


1/10 -> Train loss: 1.9546649016439914	Validation loss: 1.4414264179766179


Validation batches: 100%|███████████████████████████████████████████████████████████████| 40/40 [00:14<00:00,  2.69it/s]


2/10 -> Train loss: 1.7599840533919633	Validation loss: 1.2159628041088582


Validation batches: 100%|███████████████████████████████████████████████████████████████| 40/40 [00:15<00:00,  2.66it/s]


3/10 -> Train loss: 1.627574315480888	Validation loss: 1.0374203854550919


Validation batches: 100%|███████████████████████████████████████████████████████████████| 40/40 [00:14<00:00,  2.68it/s]


4/10 -> Train loss: 1.5205747211817653	Validation loss: 0.8858056117314845


Validation batches: 100%|███████████████████████████████████████████████████████████████| 40/40 [00:15<00:00,  2.61it/s]

5/10 -> Train loss: 1.424874269273132	Validation loss: 0.7601377387903631


In [32]:
def predict_answer(context, question, ref_answer=None):
    inputs = TOKENIZER(question, context, max_length=Q_LEN, padding="max_length", truncation=True, add_special_tokens=True)
    
    input_ids = torch.tensor(inputs["input_ids"], dtype=torch.long).to(DEVICE).unsqueeze(0)
    attention_mask = torch.tensor(inputs["attention_mask"], dtype=torch.long).to(DEVICE).unsqueeze(0)

    outputs = MODEL.generate(input_ids=input_ids, attention_mask=attention_mask)
  
    predicted_answer = TOKENIZER.decode(outputs.flatten(), skip_special_tokens=True)
    
    if ref_answer:
        # Load the Bleu metric
        bleu = evaluate.load("google_bleu")
        rouge = evaluate.load('rouge')
        bert = evaluate.load("bertscore")
        score = bleu.compute(predictions=[predicted_answer], 
                            references=[ref_answer])
        bert_score = bert.compute(predictions=[predicted_answer], references=[ref_answer], lang='en', device='cuda:1')
        
        # score_rouge = rouge.compute(predictioins=[predicted_answer],
                                   # references = [ref_answer])
    
        # print("Context: \n", context)
        # |print("\n")
        print("Question: \n", question)
        return { "Reference Answer: ": ref_answer, 
            "Predicted Answer: ": predicted_answer, 
            "BLEU Score: ": score,
            "Bert Score: ": bert_score
        }
    else:
        return predicted_answer

In [33]:
_all = {}
arr = [{} for i in range(3)]

for i in range(100):
    print('--------', i)
    question = validation_dataset.iloc[i]['generated_que']
    context = validation_dataset.iloc[i]['article']
    answer = ' '.join(validation_dataset.iloc[i]['spoiler'])
        
    print(validation_dataset.iloc[i]['labels'])
    ans = predict_answer(context, question, answer)
    _all['bleu'] = _all.get('bleu', 0) + ans["BLEU Score: "]['google_bleu']
    _all['bert'] = _all.get('bert', 0) + ans['Bert Score: ']['precision'][0]


    type = validation_dataset.iloc[i]['labels']
    arr[type]['bleu'] = arr[type].get('bleu',0) + ans["BLEU Score: "]['google_bleu']
    arr[type]['bert'] = arr[type].get('bert',0) + ans["Bert Score: "]['precision'][0]
        
        
    
    print(ans)

print(" All " + str(_all['bert']/100))
print(" Phrase " + str(arr[0]['bert']/100))
print(" Passage " + str(arr[1]['bert']/100))
print(" Multi " + str(arr[2]['bert']/100))


    

-------- 0
1


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Question: 
 What is the reason for the delay of the sequel to Five Nights at Freddy's?
{'Reference Answer: ': 'some of the plot elements are so disturbing that they are making him feel sick', 'Predicted Answer: ': 'too dark', 'BLEU Score: ': {'google_bleu': 0.0}, 'Bert Score: ': {'precision': [0.8500030040740967], 'recall': [0.8265589475631714], 'f1': [0.8381170630455017], 'hashcode': 'roberta-large_L17_no-idf_version=0.3.12(hug_trans=4.41.0.dev0)'}}
-------- 1
2


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Question: 
 What is the word that could determine the fate of Arizona Sheriff Joe Arpaio?
{'Reference Answer: ': '"intentionally" could transform a court case against Phoenix-area Sheriff Joe Arpaio from civil charges to a criminal prosecution', 'Predicted Answer: ': '"intentionally"', 'BLEU Score: ': {'google_bleu': 0.08108108108108109}, 'Bert Score: ': {'precision': [0.9389325976371765], 'recall': [0.8278279304504395], 'f1': [0.8798867464065552], 'hashcode': 'roberta-large_L17_no-idf_version=0.3.12(hug_trans=4.41.0.dev0)'}}
-------- 2
0


...

Question: 
 What is the disturbing reason?
{'Reference Answer: ': 'They are soaked in a bath of chlorine', 'Predicted Answer: ': 'They could make you very ill', 'BLEU Score: ': {'google_bleu': 0.038461538461538464}, 'Bert Score: ': {'precision': [0.8475748300552368], 'recall': [0.8505369424819946], 'f1': [0.8490532636642456], 'hashcode': 'roberta-large_L17_no-idf_version=0.3.12(hug_trans=4.41.0.dev0)'}}
-------- 98
0


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Question: 
 What is the iPhone 5C number that Apple doesn't want you to know?
{'Reference Answer: ': 'preorder figure was 2 million', 'Predicted Answer: ': 'number', 'BLEU Score: ': {'google_bleu': 0.0}, 'Bert Score: ': {'precision': [0.8293511271476746], 'recall': [0.7952939867973328], 'f1': [0.8119655847549438], 'hashcode': 'roberta-large_L17_no-idf_version=0.3.12(hug_trans=4.41.0.dev0)'}}
-------- 99
0


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Question: 
 What is the reason the mother glued pennies to her daughter's shoes?
{'Reference Answer: ': 'make-shift tap shoes', 'Predicted Answer: ': 'Knowing her daughter’s love for dancing and playing, she decided to take some of her old', 'BLEU Score: ': {'google_bleu': 0.0}, 'Bert Score: ': {'precision': [0.8247225284576416], 'recall': [0.8150407671928406], 'f1': [0.8198530673980713], 'hashcode': 'roberta-large_L17_no-idf_version=0.3.12(hug_trans=4.41.0.dev0)'}}
 All 0.8919407784938812
 Phrase 0.4297121506929398
 Passage 0.290065113902092
 Multi 0.17216351389884949
